In [1]:
# Importing required libraries
from urllib.request import urlopen
import re
import pandas as pd

In [2]:
# opening the IMDb URL using urlopen

# url = "https://www.imdb.com/chart/top/"
url = ("https://www.imdb.com/search/title?genres=sci-fi&" + "start=" + str(51) + "&explore=title_type,genres&ref_=adv_prv") #, headers=headers)

page = urlopen(url)

In [3]:
# Printing the HTTP response
print(page)

In [4]:
# Reading and converting the bytes into string (utf-8) format

bytes = page.read()
html = bytes.decode("utf-8")

In [5]:
# Finding number of movies in the html

movies = re.findall('<img alt="(.*)"', html, re.IGNORECASE)

print(len(movies))

50


In [6]:
print(html)



<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">




        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
        <title>Sci-Fi
(Sorted by Popularity Ascending) - IMDb</title>
  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>

        <link rel="canonical" href="https://www.imdb.com/search/title/?genres=sci-fi&start=51" />
        <meta 

In [7]:
# Extracting all the movies info patch in the html and storing it in a list using regex

movie_info_html = re.findall('> <img alt=.*?<div class="lister-item-image float-left">', html,re.DOTALL)
# print(movie_info_html)
print(type(movie_info_html))

<class 'list'>


In [8]:
# for i in range (len(movie_info_html)):
#     print(movie_info_html[i])
#     print("\n \n \n \n \n")

# print(movie_info_html[-1])
print(type(movie_info_html[0]))

<class 'str'>


In [9]:
# The last entry Umbrella Academy did not fall into the previous regex pattern, so scrapping it separately.
ua = re.findall('> <img alt="The Umbrella Academy".*?>David Castañeda</a>', html, re.DOTALL)

# print(ua)
movie_info_html.append(str(ua))
print(type(ua))
print(len(movie_info_html))

<class 'list'>
50


In [19]:
# sample = movie_info_html[-1]

# print(type(movie_info_html[-1]))

sample = str(ua)
print(sample)

['> <img alt="The Umbrella Academy"\nclass="loadlate"\nloadlate="https://m.media-amazon.com/images/M/MV5BOTdlODM0MTAtMzRiZi00MTQxLWE0MGUtNTNjOGZjNjAzN2E0XkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UY98_CR5,0,67,98_AL_.jpg"\ndata-tconst="tt1312171"\nheight="98"\nsrc="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png"\nwidth="67" />\n</a>        </div>\n        <div class="lister-item-content">\n<h3 class="lister-item-header">\n        <span class="lister-item-index unbold text-primary">100.</span>\n    \n    <a href="/title/tt1312171/"\n>The Umbrella Academy</a>\n    <span class="lister-item-year text-muted unbold">(2019–2024)</span>\n</h3>\n    <p class="text-muted ">\n            <span class="certificate">TV-14</span>\n                 <span class="ghost">|</span> \n                <span class="runtime">60 min</span>\n                 <span class="ghost">|</span> \n            <span class="genre">\nAction, Adventure, Comedy            </span>\n    </p>\n    <div class="ratings-bar">\n

In [27]:
movie_genre = re.findall('Action.*?Comdedy', str(ua))
print(movie_genre)

[]


In [21]:
# Exrtracting details of the movies to create a database

# Extracting movie title
movie_name = re.findall('> <img alt="(.*?)"', sample)
print(movie_name)

# Extracting the Genre
movie_genre = re.findall('"genre">\n(.*?)            </span>\n', sample, re.DOTALL)
print(movie_genre)

# Extracting IMDb movie Rating
movie_rating = re.findall('ratingValue" content="(.*?)"', sample)
print(movie_rating)

# Extracting top IMDb rating by a user
movie_top_rating = re.findall('bestRating" content="(.*?)"', sample)
print(movie_top_rating)

# Extracting the total number of user ratings
movie_rating_count = re.findall('ratingCount" content="(.*?)"', sample)
print(movie_rating_count)

# Extracting movie description
movie_desc = re.findall('<p class="text-muted">\n(.*?)</p>', sample)
print(movie_desc)

# Extracting Director name
ext = re.findall('Director:\n<a href="/name/nm.*?</a>', sample, re.DOTALL)
# print(ext)
if len(ext) > 0:
    movie_director = re.findall('>(.*?)</a>', ext[0])
else:
    movie_director = []
print(movie_director)

# Extracting Actors information
ext_st = re.findall('Stars:.*?</p>', sample, re.DOTALL)
print(ext_st)
# movie_actors = re.findall('>(.*?)</a>', ext_st[0])
# print(movie_actors)

['The Umbrella Academy']
[]
['7.9']
['10']
['265636']
[]
[]
[]


In [50]:
# Creating a function to Extract info of the movies and tv shows

def scraper(sample):
    # Exrtracting details of the movies to create a database

    # Extracting movie title
    movie_name = re.findall('> <img alt="(.*?)"', sample)

    # Extracting the Genre
    movie_genre = re.findall('genre">\n(.*?)            </span>', sample, re.DOTALL)

    # Creating a list of genre if the movie or tv_show has more than one Genre
    if ',' in movie_genre[0]:
        movie_genre = movie_genre[0].split(', ')

    # Extracting IMDb movie Rating
    movie_rating = re.findall('ratingValue" content="(.*?)"', sample)

    # Extracting top IMDb rating by a user
    movie_top_rating = re.findall('bestRating" content="(.*?)"', sample)

    # Extracting the total number of user ratings
    movie_rating_count = re.findall('ratingCount" content="(.*?)"', sample)

    # Extracting movie description
    movie_desc = re.findall('<p class="text-muted">\n(.*?)</p>', sample)

    # Extracting Director name
    ext = re.findall('Director:\n<a href="/name/nm.*?</a>', sample, re.DOTALL)

    # Multiple extractions required so checking if it has a value or not
    if len(ext) > 0:
        movie_director = re.findall('>(.*?)</a>', ext[0])
    else:
        movie_director = []

    # Extracting Actors information
    ext_st = re.findall('Stars:.*?</p>', sample, re.DOTALL)

    if len(ext_st) > 0:
        movie_actors = re.findall('>(.*?)</a>', ext_st[0])
    else:
        movie_actors = []

    contents = {
        "Title": movie_name,
        "Genre": movie_genre,
        "Rating": movie_rating,
        "Best_Rating": movie_top_rating,
        "No_Of_Ratings": movie_rating_count,
        "Storyline": movie_desc,
        "Director": movie_director,
        "Cast": movie_actors
    }
    return contents

In [51]:
# Testing the updated Scraper function

test = scraper(movie_info_html[0])
print(test)

{'Title': ['Adventure Time'], 'Genre': ['Animation', 'Action', 'Adventure'], 'Rating': ['8.6'], 'Best_Rating': ['10'], 'No_Of_Ratings': ['106990'], 'Storyline': ['A 12-year-old boy and his best friend, a wise 28-year-old dog with magical powers, go on a series of surreal adventures in a remote future.'], 'Director': [], 'Cast': ['John DiMaggio', 'Jeremy Shada', 'Tom Kenny', 'Hynden Walch']}


In [58]:
# Converting the entire scraped data (HTML list of movies) into a database

# Creating an empty DataFrame
df = pd.DataFrame(columns=['Title', 'Genre', 'Rating', 'Best_Rating', 'No_Of_Ratings', 'Storyline', 'Director', 'Cast'])

for data in movie_info_html:
    try:
        test = scraper(data)
    except Exception as e:
        print(f"Error Occured {e} for {data}")

Error Occured expected string or bytes-like object for ['> <img alt="The Umbrella Academy"\nclass="loadlate"\nloadlate="https://m.media-amazon.com/images/M/MV5BOTdlODM0MTAtMzRiZi00MTQxLWE0MGUtNTNjOGZjNjAzN2E0XkEyXkFqcGdeQXVyMTkxNjUyNQ@@._V1_UY98_CR5,0,67,98_AL_.jpg"\ndata-tconst="tt1312171"\nheight="98"\nsrc="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png"\nwidth="67" />\n</a>        </div>\n        <div class="lister-item-content">\n<h3 class="lister-item-header">\n        <span class="lister-item-index unbold text-primary">100.</span>\n    \n    <a href="/title/tt1312171/"\n>The Umbrella Academy</a>\n    <span class="lister-item-year text-muted unbold">(2019–2024)</span>\n</h3>\n    <p class="text-muted ">\n            <span class="certificate">TV-14</span>\n                 <span class="ghost">|</span> \n                <span class="runtime">60 min</span>\n                 <span class="ghost">|</span> \n            <span class="genre">\nAction, Adventure, Comedy       

In [31]:
Extracted_Information = (map(scraper, movie_info_html))

In [35]:
# print(type(Extracted_Information))

for i in Extracted_Information:
    print(i)

In [10]:
df = pd.DataFrame(movies, columns=['Title'])

In [14]:
df.describe()

,Title
count,50
unique,49
top,The Flash
freq,2
